# **Isolation Forest**



## 0.환경준비

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest # Isolation Forest!
from sklearn.metrics import *

from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore')

## 1.Isolation Forest 개념이해

> sklearn help : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html  
Isolation Forest Original Paper : https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest

### (1) 샘플데이터 가져오기

In [ ]:
# Single Blob
X1 = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/Anomaly_X.csv')

# Double Blob
X2 = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/Anomaly_X2.csv')

### (2) 모델 시각화 함수

In [ ]:
def model_visualize(model, v1, v2, title = "") :
    # 메쉬그리드값 저장하기
    xx, yy = np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50)) # mesh grid

    # 메쉬 그리드값에 대해 모델 부터 Anomaly Score 만들기.
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()]) # Anomaly Score
    Z = Z.reshape(xx.shape)
    # 시각화
    plt.figure(figsize = (8,8))
    plt.title(title)

    # 메쉬그리드 값의 Anomaly Score에 대한 등고선
    plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

    # 데이터 산점도 그리기.(예측 결과 Abnormal은 오렌지색, Normal은 흰색)
    sns.scatterplot(x=v1, y=v2, sizes = 30, edgecolor='k', hue = pred, palette=['white', 'orange'])

    plt.axis("tight")
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.show()

### (3) Single Blob

In [ ]:
# sample data
plt.figure(figsize = (8,8))

plt.scatter(X1['v1'], X1['v2'], c="white", s=30, edgecolor="k")

plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid()
plt.show()

* Isolation Forest 모델을 일단 만들어 봅시다.

In [ ]:
# 모델링
model = IsolationForest(contamination = 0.1, n_estimators = 50 )
model.fit(X1)
pred = model.predict(X1)
pred

In [ ]:
pred = np.where(pred == 1, 0, 1)
pred

* 모델을 시각화 해 봅시다.

In [ ]:
model_visualize(model, X1['v1'], X1['v2'], 'Isolation Forest')

* 몇가지를 조절해 봅시다.
    * contamination
        * Score 계산 후, 이상치로 간주할 비율
    * n_estimators
        * 하나의 데이터를 고립시키기 위해 생성하는 tree의 개수
        * Default : 100

In [ ]:
# 여기를 조정해 봅시다.
cont = 0.4
n_est = 100

# 모델링
model = IsolationForest(contamination = cont, n_estimators = n_est)
model.fit(X1)
pred = model.predict(X1)
pred = np.where(pred == 1, 0, 1)

model_visualize(model, X1['v1'], X1['v2'], 'Isolation Forest')

### (4) Double Blob

In [ ]:
plt.figure(figsize = (8,8))

plt.scatter(X2['v1'], X2['v2'], c="white", s=30, edgecolor="k")

plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid()
plt.show()

* Isolation Forest 모델을 일단 만들어 봅시다.

In [ ]:
# 모델링
model = IsolationForest(contamination = 0.1, n_estimators = 50 )
model.fit(X2)
pred = model.predict(X2)
pred = np.where(pred == 1, 0, 1)

* 모델을 시각화 해 봅시다.

In [ ]:
model_visualize(model, X2['v1'], X2['v2'], 'Isolation Forest')

* 몇가지를 조절해 봅시다.
    * contamination
        * 데이터에서 존재하는 Abnormal 데이터의 비율.
        * Score 계산 후, 이상치로 간주할 비율
    * n_estimators
        * 하나의 데이터를 고립시키기 위해 생성하는 tree의 개수
        * Default : 100

In [ ]:
# 여기를 조정해 봅시다.
cont = 0.1
n_est = 100

# 모델링
model = IsolationForest(contamination = cont, n_estimators = n_est)
model.fit(X2)
pred = model.predict(X2)
pred = np.where(pred == 1, 0, 1)

# 모델 시각화
model_visualize(model, X2['v1'], X2['v2'], 'Isolation Forest')

## 2.🤔[실습]Semiconductor manufacturing process dataset


![](https://assets.pandaily.com/uploads/2021/10/semiconductor.png)

* 반도체 제조 공정은 시점별로 수많은 센서로부터 정보를 수집하여 공정을 감시합니다.
* 센서정보와 함께 공정간 발생된 불량품에 대한 정보를 저장하였습니다.
* 불량을 예측해 봅시다.

### (1) 데이터 준비

* 데이터 로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)
data.head()

변수 정보
* label : 1 - 불량, 0 - 정상
* v### : 센서값들


In [ ]:
target = 'label'

In [ ]:
data[target].value_counts()

In [ ]:
data[target].value_counts(normalize = True)

* x, y로 나누기

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 데이터 분할
    * 이미 test set은 분할되어 있다고 가정합니다.
    * 주어진 데이터를 train set : validation set 으로 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state = 20)

In [ ]:
print(x_train.shape, x_val.shape)

### (2) 모델링
* contamination를 y_train의 Abnormal 비율로 지정하여 학습하시오.

In [ ]:
# y의 비율 찾기


#### 1)모델링

In [ ]:
# contamination=0.071로 지정하고 모델을 생성해 봅시다.


# 비지도 학습입니다. x_train만 사용!


#### 2)예측 및 평가

In [ ]:
# 예측


# 결과를 1,0으로 변환(np.where)


# 분류 결과를 평가해 봅시다.
